# Run MLFlow Model in Seldon Core

This notebook shows how you can easily train a model using [MLFlow](https://mlflow.org/) and serve requests within Seldon Core on Kubernetes.

Dependencies

  * ```pip install seldon-core```
  * ```pip install mlflow```
  


## Train Example MlFlow Model

In [8]:
!git clone https://github.com/mlflow/mlflow

Cloning into 'mlflow'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 7786 (delta 3), reused 4 (delta 1), pack-reused 7772
Receiving objects: 100% (7786/7786), 8.15 MiB | 407.00 KiB/s, done.
Resolving deltas: 100% (4781/4781), done.
Checking connectivity... done.


In [9]:
!python mlflow/examples/sklearn_elasticnet_wine/train.py

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689


## Test Inference Locally

In [3]:
!pygmentize MyMlflowModel.py

from mlflow import pyfunc
import os
import pandas as pd

class MyMlflowModel(object):

    def __init__(self):
        self.pyfunc_model = pyfunc.load_pyfunc("mlruns/0/"+next(os.walk('mlruns/0'))[1][0]+"/artifacts/model")
        
    def predict(self,X,features_names):
        if not features_names is None and len(features_names)>0:
            df = pd.DataFrame(data=X,columns=features_names)
        else:
            df = pd.DataFrame(data=X)
        return self.pyfunc_model.predict(df)


In [10]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.13 mlflow_model:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-ex

  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Stored in directory: /root/.cache/pip/wheels/bc/a5/8d/7d021f2741ed8a7354e4fe19c1890e71ed719d681cd16cf292
Stored in directory: /root/.cache/pip/wheels/7a/3e/18/4770e5048530a759970f03239d88b3d32d7434c410d803cfc9
Stored in directory: /root/.cache/pip/wheels/ee/09/99/bf937e4f02788fa8b33dc5240842b

In [11]:
!docker run --name "mlflow_model" -d --rm -p 5000:5000 mlflow_model:0.1

44ba8b3fd0999714f4a56f061275973a54564aee4e1161c79e7fbb8288adea9d


In [13]:
!curl -H "Content-Type: application/x-www-form-urlencoded" -g 0.0.0.0:5000/predict -d 'json={"data":{"names":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"ndarray":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}}'

{"data":{"ndarray":[5.455573233630147]},"meta":{}}


In [14]:
!curl -H "Content-Type: application/x-www-form-urlencoded" -g 0.0.0.0:5000/predict -d 'json={"data":{"ndarray":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}}'

{"data":{"ndarray":[5.455573233630147]},"meta":{}}


In [15]:
!docker rm mlflow_model --force

mlflow_model


## Test Inference on Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [9]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


## Setup Seldon Core

Follow the instructions to [Setup Cluster](../../seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](../../seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](../../seldon_core_setup.ipynb#Install-Seldon-Core).

In [16]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.13 mlflow_model:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-ex

  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
  Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
Stored in directory: /root/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a1

In [17]:
!kubectl create -f deployment.json

seldondeployment.machinelearning.seldon.io/mlflow-example created


In [18]:
!kubectl rollout status deployment/mlflow-dep-mlflow-pred-d580056

Waiting for deployment "mlflow-dep-mlflow-pred-d580056" rollout to finish: 0 of 1 updated replicas are available...
deployment "mlflow-dep-mlflow-pred-d580056" successfully rolled out


In [19]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    mlflow-example --namespace seldon -p

----------------------------------------
SENDING NEW REQUEST:

[[2.8040e+00 7.5600e-01 2.8000e-02 1.3500e-01 5.1000e-02 6.6000e-02
  6.8270e+00 8.6900e-01 2.0100e-01 7.5268e+01 4.2500e-01]]
RECEIVED RESPONSE:
meta {
  puid: "fjmht64oou6r5f8b8ju3aagu4u"
  requestPath {
    key: "model"
    value: "mlflow_model:0.1"
  }
}
data {
  ndarray {
    values {
      number_value: 5.1800195352034475
    }
  }
}




In [ ]:
!minikube delete